In [ ]:
!pip install ultralytics

In [ ]:
!pip uninstall -y opencv-python opencv-contrib-python opencv-python-headless

In [ ]:
!pip install pycocotools

In [ ]:
!pip install opencv-python-headless==4.12.0.88

In [ ]:
!pip install onnxruntime-gpu

In [ ]:
!pip install onnxruntime

In [ ]:
!pip install nvidia-cuda-runtime-cu12 nvidia-cublas-cu12 nvidia-cudnn-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-cufft-cu12 nvidia-curand-cu12

In [ ]:
!export LD_LIBRARY_PATH="$(python -c 'import site,glob; import sys; print(":".join([p for d in site.getsitepackages() for p in glob.glob(d+"/nvidia/*/lib")]))')":${LD_LIBRARY_PATH}

In [ ]:
import os
import shutil

import os
import zipfile
import urllib.request

In [ ]:
datasets_dir = os.environ['DOMINO_DATASETS_DIR']
project_ds_folder = os.environ['DOMINO_PROJECT_NAME'] 

download_base_folder=f"{datasets_dir}/{project_ds_folder}"
models_folder = "models"
yolo_model_name="yolov8n"

yolo_onnx_file_name=f"{yolo_model_name}.onnx"

In [ ]:
from ultralytics import YOLO
def download_yolo(download_base_folder,models_folder,yolo_model_name):    
    model = YOLO(yolo_model_name)  # or yolov8s.pt, yolov8m.pt, yolov8l.pt
    model.export(format="onnx")
    yolo_file_name=f"{yolo_model_name}.pt"
    shutil.move(yolo_onnx_file_name,f"{download_base_folder}/{models_folder}/{yolo_onnx_file_name}")
    os.remove(yolo_file_name)

In [ ]:
download_yolo(download_base_folder,models_folder,yolo_model_name)

In [ ]:
import os, urllib.request, zipfile, shutil
from pycocotools.coco import COCO

def download_coco_2017_val_ds(base_dir):
    os.makedirs(base_dir, exist_ok=True)

    img_set = "val2017"
    images_url = f"http://images.cocodataset.org/zips/{img_set}.zip"
    ann_url    = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

    images_zip = os.path.join(base_dir, f"{img_set}.zip")
    ann_zip    = os.path.join(base_dir, "annotations_trainval2017.zip")

    # download
    if not os.path.exists(images_zip):
        print(f"Downloading {os.path.basename(images_zip)}...")
        urllib.request.urlretrieve(images_url, images_zip)

    if not os.path.exists(ann_zip):
        print(f"Downloading {os.path.basename(ann_zip)}...")
        urllib.request.urlretrieve(ann_url, ann_zip)

    # extract images
    target_images_dir = os.path.join(base_dir, "images", img_set)
    if not os.path.exists(target_images_dir):
        print(f"Extracting {os.path.basename(images_zip)}...")
        with zipfile.ZipFile(images_zip, "r") as zf:
            zf.extractall(base_dir)  # creates base_dir/val2017
        os.makedirs(os.path.join(base_dir, "images"), exist_ok=True)
        src = os.path.join(base_dir, img_set)  # base_dir/val2017
        if os.path.exists(src):
            shutil.move(src, target_images_dir)
        else:
            # already extracted to correct place by prior run
            os.makedirs(target_images_dir, exist_ok=True)

    # extract annotations
    ann_dir = os.path.join(base_dir, "annotations")
    ann_json = os.path.join(ann_dir, f"instances_{img_set}.json")
    if not os.path.exists(ann_json):
        print(f"Extracting {os.path.basename(ann_zip)}...")
        with zipfile.ZipFile(ann_zip, "r") as zf:
            zf.extractall(base_dir)  # creates base_dir/annotations/*.json
    assert os.path.exists(ann_json), f"Missing {ann_json}"

    # build labels
    labels_dir = os.path.join(base_dir, "labels", img_set)
    if not os.path.exists(labels_dir):
        print(f"Converting COCO → YOLO labels into {labels_dir} ...")
        os.makedirs(labels_dir, exist_ok=True)

        coco = COCO(ann_json)
        cats = coco.loadCats(coco.getCatIds())
        # COCO category ids are not 0..79; map to contiguous 0-based
        catid2cls = {c["id"]: i for i, c in enumerate(cats)}

        img_ids = coco.getImgIds()
        for img in coco.loadImgs(img_ids):
            w, h = img["width"], img["height"]
            ann_ids = coco.getAnnIds(imgIds=img["id"], iscrowd=False)
            anns = coco.loadAnns(ann_ids)

            lines = []
            for a in anns:
                x, y, bw, bh = a["bbox"]
                if bw <= 0 or bh <= 0:
                    continue
                x_c = (x + bw / 2) / w
                y_c = (y + bh / 2) / h
                bw_n = bw / w
                bh_n = bh / h
                cls = catid2cls[a["category_id"]]
                lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {bw_n:.6f} {bh_n:.6f}")

            label_path = os.path.join(labels_dir, os.path.splitext(img["file_name"])[0] + ".txt")
            with open(label_path, "w") as f:
                f.write("\n".join(lines))

    print("COCO 2017 val images, annotations, and YOLO labels are ready.")


In [ ]:
base_coco_dir = f"{download_base_folder}/coco"

In [ ]:
prepare_coco_yolo(base_coco_dir)

In [ ]:
model_path = f"{download_base_folder}/{models_folder}/{yolo_onnx_file_name}"
print(model_path)
base_folder=f"{download_base_folder}/coco"


In [ ]:
domino_user_name = os.environ['DOMINO_USER_NAME']
experiment_name=f"cv-comparison-{domino_user_name}"
metrics = evaluate_model(f"{download_base_folder}/coco",model_path,limit_images=10,experiment_name=experiment_name,device="gpu")


In [ ]:
print(metrics)
